# Agen AI Azure dengan Dukungan Model Context Protocol (MCP) - Python

Notebook ini menunjukkan cara menggunakan Agen AI Azure dengan alat Model Context Protocol (MCP) dalam Python. Ini memperlihatkan cara membuat agen cerdas yang dapat memanfaatkan server MCP eksternal (seperti Microsoft Learn) untuk kemampuan yang lebih baik menggunakan autentikasi tanpa kunci.


## Instal Paket Python yang Diperlukan

Pertama, kita perlu menginstal paket Python yang diperlukan:
- **azure-ai-projects**: SDK inti untuk Azure AI Projects
- **azure-ai-agents**: SDK Azure AI Agents untuk membuat dan mengelola agen
- **azure-identity**: Menyediakan autentikasi tanpa kunci menggunakan DefaultAzureCredential
- **mcp**: Implementasi Model Context Protocol untuk Python


## Manfaat Autentikasi Tanpa Kunci

Notebook ini menunjukkan **autentikasi tanpa kunci** yang memberikan beberapa keuntungan:
- ✅ **Tidak ada API key yang perlu dikelola** - Menggunakan autentikasi berbasis identitas Azure
- ✅ **Keamanan yang lebih baik** - Tidak ada rahasia yang disimpan dalam kode atau file konfigurasi
- ✅ **Rotasi kredensial otomatis** - Azure menangani manajemen siklus hidup kredensial
- ✅ **Kontrol akses berbasis peran** - Menggunakan Azure RBAC untuk izin yang lebih terperinci
- ✅ **Dukungan multi-lingkungan** - Berfungsi dengan lancar di lingkungan pengembangan dan produksi

`DefaultAzureCredential` secara otomatis memilih sumber kredensial terbaik yang tersedia:
1. **Managed Identity** (saat berjalan di Azure)
2. Kredensial **Azure CLI** (selama pengembangan lokal)
3. Kredensial **Visual Studio**
4. **Variabel lingkungan** (jika dikonfigurasi)
5. Autentikasi **browser interaktif** (sebagai cadangan)


## Pengaturan Autentikasi Tanpa Kunci

**Prasyarat untuk autentikasi tanpa kunci:**

### Untuk Pengembangan Lokal:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Untuk Lingkungan Azure:
- Aktifkan **System-assigned Managed Identity** pada sumber daya Azure Anda
- Tetapkan **peran RBAC** yang sesuai untuk managed identity:
  - `Cognitive Services OpenAI User` untuk akses Azure OpenAI
  - `AI Developer` untuk akses Proyek AI Azure

### Variabel Lingkungan (Opsional):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Tidak perlu kunci API atau string koneksi!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Impor Perpustakaan yang Diperlukan

Impor modul Python yang diperlukan:
- **os, time**: Perpustakaan standar Python untuk variabel lingkungan dan penundaan
- **AIProjectClient**: Klien utama untuk Azure AI Projects
- **DefaultAzureCredential**: Autentikasi tanpa kunci untuk layanan Azure
- **Kelas terkait MCP**: Untuk membuat dan mengelola alat MCP serta menangani persetujuan


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfigurasi Pengaturan Server MCP

Atur konfigurasi server MCP menggunakan variabel lingkungan dengan nilai default sebagai cadangan:
- **MCP_SERVER_URL**: URL server MCP (defaultnya adalah Microsoft Learn API)
- **MCP_SERVER_LABEL**: Label untuk mengidentifikasi server MCP (defaultnya adalah "mslearn")

Pendekatan ini memungkinkan konfigurasi yang fleksibel di berbagai lingkungan.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Membuat Klien Proyek Azure AI (Autentikasi Tanpa Kunci)

Inisialisasi klien Proyek Azure AI menggunakan **autentikasi tanpa kunci**:
- **endpoint**: URL endpoint proyek Azure AI Foundry
- **credential**: Menggunakan `DefaultAzureCredential()` untuk autentikasi yang aman tanpa kunci
- **Tidak memerlukan kunci API**: Secara otomatis mendeteksi dan menggunakan kredensial terbaik yang tersedia

**Alur Autentikasi:**
1. Memeriksa Managed Identity (di lingkungan Azure)
2. Beralih ke kredensial Azure CLI (untuk pengembangan lokal)
3. Menggunakan sumber kredensial lain yang tersedia jika diperlukan

Pendekatan ini menghilangkan kebutuhan untuk mengelola kunci API atau string koneksi dalam kode Anda.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Membuat Definisi Alat MCP

Buat alat MCP yang terhubung ke server Microsoft Learn MCP:
- **server_label**: Pengenal untuk server MCP
- **server_url**: URL endpoint dari server MCP
- **allowed_tools**: Daftar opsional untuk membatasi alat yang dapat digunakan (daftar kosong mengizinkan semua alat)

Alat ini akan memungkinkan agen untuk mengakses dokumentasi dan sumber daya Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Membuat Agen dan Melakukan Percakapan (Alur Kerja Tanpa Kunci)

Bagian ini menjelaskan secara lengkap **alur kerja agen tanpa kunci**:

1. **Membuat AI Agent**: Siapkan agen dengan model GPT-4.1 nano dan alat MCP
2. **Membuat Thread**: Buat thread percakapan untuk komunikasi
3. **Mengirim Pesan**: Tanyakan kepada agen tentang perbedaan Azure OpenAI dan OpenAI
4. **Menangani Persetujuan Alat**: Secara otomatis menyetujui panggilan alat MCP jika diperlukan
5. **Memantau Eksekusi**: Lacak kemajuan agen dan tangani tindakan yang diperlukan
6. **Menampilkan Hasil**: Tunjukkan detail percakapan dan penggunaan alat

**Fitur Tanpa Kunci:**
- ✅ **Tidak ada rahasia yang dikodekan secara langsung** - Semua autentikasi ditangani oleh identitas Azure
- ✅ **Aman secara default** - Menggunakan kontrol akses berbasis peran
- ✅ **Penyebaran yang disederhanakan** - Tidak diperlukan pengelolaan kredensial
- ✅ **Ramah audit** - Semua akses dilacak melalui identitas Azure

Agen akan menggunakan alat MCP untuk mengakses sumber daya Microsoft Learn dengan keamanan penuh tanpa pengelolaan kunci API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
